## Importation des bibliothèques

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()  
import pickle  
import warnings  
warnings.filterwarnings("ignore")
import json 

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## Traitement du fichier json

In [2]:
with open('intentions.json', encoding='utf-8') as json_data:
    intentions = json.load(json_data)
mots = []
classes = []
documents = []
ignore_mots = ['?']
for intention in intentions['intentions']:
    for pattern in intention['patterns']:
        m = nltk.word_tokenize(pattern)  
        mots.extend(m)

        documents.append((m, intention['tag'])) 
        if intention['tag'] not in classes:
            classes.append(intention['tag'])
mots = [stemmer.stem((m.lower())) for m in mots if m not in ignore_mots]
mots = sorted(list(set(mots)))
classes = sorted(list(set(classes)))

## Création des données pour le model

In [3]:
training = []
output = []
output_vide = [0] * len(classes)
for doc in documents:
    ensemble = []
    pattern_mots = doc[0]
    pattern_mots = [stemmer.stem(mot.lower()) for mot in pattern_mots]

    for m in mots:
        ensemble.append(1) if m in pattern_mots else ensemble.append(0)
    output_row = list(output_vide)
    output_row[classes.index(doc[1])] = 1
    training.append([ensemble, output_row])
training = np.array(training)

## Création de listes de test et entrainement

In [4]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

## Création du réseau de neurone et entrainement du model

In [5]:
tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
# print("training.....")
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

model.save('model.teflearn')

Training Step: 8999  | total loss: 0.06456 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.06456 - acc: 0.9612 -- iter: 64/71
Training Step: 9000  | total loss: 0.05859 | time: 0.039s
| Adam | epoch: 1000 | loss: 0.05859 - acc: 0.9651 -- iter: 71/71
--
INFO:tensorflow:C:\Users\oumai\Desktop\Projet Chatbot\model.teflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\oumai\Desktop\Projet Chatbot\model.teflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\oumai\Desktop\Projet Chatbot\model.teflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\oumai\Desktop\Projet Chatbot\model.teflearn.meta
INFO:tensorflow:100


## Traitement des données

In [6]:
pickle.dump({'mots': mots, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", 'wb'))



data = pickle.load(open("training_data", "rb"))
mots = data['mots']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
with open('intentions.json', encoding='utf-8') as json_data:
    intentions = json.load(json_data)

def traitement_expression(expression):
    expression_mots = nltk.word_tokenize(expression)
    expression_mots = [stemmer.stem(mot.lower()) for mot in expression_mots]
    return expression_mots


def ensemble_tab(expression, mots, show_details=False):
    expression_mots = traitement_expression(expression)
    ensemble = [0] * len(mots)
    for e in expression_mots:
        for i, m in enumerate(mots):
            if m == e:
                ensemble[i] = 1
                if show_details:
                    print("c'est trouvé dans l'ensemble: %e" % m)
    return (np.array(ensemble))

def classification(expression):
    results = model.predict([ensemble_tab(expression, mots)])[0] 
    results = [[i, r] for i, r in enumerate(results)]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list 

In [ ]:
import random
def response(expression, ID_Utilisateur='123', show_details=False):
    results = classification(expression)
    if results:
        while results:
            for i in intentions['intentions']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)
            print(results)
while True:
    input_data = input("Question:")
    answer = response(input_data)
    answer

Question:Bonjour
Salut, comment je peux t'aider
Question:quelles sont les filières existantes à l'école?
pour cycle ingénieur, master ou doctorat?
Question:pour cycle ingénieur
il y a 3 filières ICSD, ISITD et IIN
Question:comment je peux contacter un lauréat?
visiter le site esi.ac.ma puis Nos lauréats
Question:Merci
avec plaisir
Question:How are you
Everything is going well thank you
Question:are there any intership offers?
Visit internships and PFE under the heading Our students
Question:Thank you
You're welcome
